# AI Pharmacy - Pregnancy

In [1]:
from dotenv import load_dotenv
from langchain.chains.question_answering.map_rerank_prompt import output_parser
from langchain_community.chains.pebblo_retrieval.enforcement_filters import PINECONE
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_pinecone import PineconeVectorStore

load_dotenv()

True

In [ ]:
!pip install pypdf

## 데이터 로드

### csv 파일

In [10]:
from langchain_community.document_loaders import CSVLoader
dur_pregnant = CSVLoader('../data_pregnancy/DUR_pregnant.csv')
pregnancy_csv_docs = dur_pregnant.load()

### pdf 파일 -- TextSplitter


In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# 폴더 경로
folder_path = '../data_pregnancy'

# PDF 파일 경로 list
pdf_paths = [
    os.path.join(folder_path, file)
    for file in os.listdir(folder_path)
    if file.startswith('pregnancy_') and file.endswith('.pdf')
]
# print(pdf_paths)

# pdf 문서 저장 list
pregnancy_pdf_docs = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load() # PDF의 각 페이지를 문서로 로드

    # 각 페이지의 개행 문자 제거
    for doc in docs:
        doc.page_content = doc.page_content.replace('\n', ' ')
    # 처리된 문서 >> pregnancy_pdf_docs에 추가
    pregnancy_pdf_docs.extend(docs)

# 텍스트 스플리터 설정
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # 청크 크기
    chunk_overlap=20  # 청크 간의 겹침
)
pregnancy_text_docs = splitter.split_documents(pregnancy_pdf_docs)

# 결과 확인
print(f"총 {len(pregnancy_pdf_docs)}개의 문서가 로드되었습니다.")
print(type(pregnancy_pdf_docs)) # list
print(len(pregnancy_text_docs)) # 195

총 14개의 문서가 로드되었습니다.
<class 'list'>
195


In [23]:
print(pregnancy_text_docs[10])

page_content='약물치료가 꼭 필요한지 전문가와 다시 검토하고 대증요법  등의 다른 치료방법을 우선적으로 모색해야 합니다.  7. 임신 중 질환에 따라 약물을 꼭 복용해야 하는 경우도 있습니다' metadata={'source': '../data_pregnancy\\pregnancy_medication_rules.pdf', 'page': 0}


### documents 합치기

In [24]:
documents = pregnancy_csv_docs + pregnancy_text_docs

In [25]:
print(type(documents))
print(len(documents))

<class 'list'>
19209


## 임베딩 모델 생성